In [2]:
!pip install PyMySQL

In [13]:
import sys
import pymysql
import datetime

import getNYT as gn
import getCNN as gc
import getBBC as gb

In [7]:
try:
    conn = pymysql.connect(host='10.120.28.52',
                       user='gp3',
                       password='Group_03',
                       db='gp3',
                       charset='utf8',
                       autocommit=True,
                       cursorclass=pymysql.cursors.DictCursor)
    c= conn.cursor()
except:
    print('Cannot connect to MySQL server!')

In [14]:
# CNN test
cats = gc.getCatLinks()

for cat in cats:
    okcount = 0
    ngcount = 0
#     print(cat)
    
    for article in gc.getArticlesList(cat):
        try:
            c.execute("""INSERT INTO cklist(title,source,url) VALUES (%(title)s,%(source)s,%(url)s)""",article)
            print('Insert successful: %s'%(article['url']))
            okcount += 1
        except:
            print('%s Error: %s'%(sys.exc_info()[0],article['url']))
            ngcount += 1
print('OK:%s ; NG:%s'%(okcount,ngcount))

<class 'pymysql.err.IntegrityError'> Error: http://edition.cnn.com/2017/04/17/us/cleveland-facebook-homicide-video/index.html
<class 'pymysql.err.IntegrityError'> Error: http://edition.cnn.com/2017/04/17/us/facebook-homicide-victim-trnd/index.html
<class 'pymysql.err.IntegrityError'> Error: http://money.cnn.com/2017/04/17/technology/facebook-safety-protocol/index.html
<class 'pymysql.err.IntegrityError'> Error: http://edition.cnn.com/videos/justice/2017/04/17/cleveland-facebook-killing-suspect-manhunt-expands-outside-ohio-es.cnn
<class 'pymysql.err.IntegrityError'> Error: http://edition.cnn.com/2017/04/17/opinions/facebook-shooting-death-cevallos/index.html
<class 'pymysql.err.IntegrityError'> Error: http://edition.cnn.com/2017/04/17/health/prince-death-investigation-documents/index.html
<class 'pymysql.err.IntegrityError'> Error: http://edition.cnn.com/2017/04/17/us/atlanta-i-20-road-damage-trnd/index.html
<class 'pymysql.err.IntegrityError'> Error: http://edition.cnn.com/2017/04/17/p

D:\Anaconda3_4.3.1\lib\site-packages\pymysql\cursors.py:323: Warning: (1265, "Data truncated for column 'title' at row 1")
  self._do_get_result()


Insert successful: http://edition.cnn.com/2017/02/09/health/firefighter-adopts-baby-he-delivered/index.html
Insert successful: http://edition.cnn.com/2017/01/19/health/iyw-cancer-survivor-marathon-trnd/index.html
Insert successful: http://edition.cnn.com/videos/world/2017/03/06/iyw-david-beckham-unicef-seven-fund.cnn
Insert successful: http://edition.cnn.com/2017/02/09/entertainment/iyw-jon-bon-jovi-jbj-soul-kitchen-trnd/index.html
Insert successful: http://edition.cnn.com/2016/11/02/entertainment/iyw-dolly-parton-imagination-library/index.html
Insert successful: http://edition.cnn.com/2016/09/20/us/iyw-carter-habit-for-humanity/index.html
Insert successful: http://edition.cnn.com/2016/09/12/world/iyw-jane-goodall-chimpanzees-microcredit-loan-program/index.html
Insert successful: http://edition.cnn.com/2016/09/06/entertainment/iyw-erykah-badu-detroit-rape-kits/index.html
Insert successful: http://edition.cnn.com/2016/10/27/entertainment/iyw-viola-davis-healing-project/index.html
Insert

In [ ]:
# NYT test
articles = gn.getArticlesList(365)
okcount = 0
ngcount = 0

for article in articles:
#     print(cat)
    try:
        c.execute("""INSERT INTO cklist(title,source,url) VALUES (%(title)s,%(source)s,%(url)s)""",article)
        print('Insert successful: %s'%(article['url']))
        okcount += 1
    except:
        print('%s Error: %s'%(sys.exc_info()[0],article['url']))
        ngcount += 1
print('OK:%s ; NG:%s'%(okcount,ngcount))

In [12]:
# BBC test
cats = gb.getCatLinks()
okcount = 0
ngcount = 0

for cat in cats:
#     print(cat)
    for article in gb.getArticlesList(cat):
        try:
            c.execute("""INSERT INTO cklist(title,source,url) VALUES (%(title)s,%(source)s,%(url)s)""",article)
            print('Insert successful: %s'%(article['url']))
            okcount += 1
        except:
            print('%s Error: %s'%(sys.exc_info()[0],article['url']))
            ngcount += 1
print('OK:%s ; NG:%s'%(okcount,ngcount))

Insert successful: http://www.bbc.com/news/world-europe-39626116
Insert successful: http://www.bbc.com/news/world-asia-39623882
Insert successful: http://www.bbc.com/news/world-asia-39626131
Insert successful: http://www.bbc.com/news/world-europe-39625509
Insert successful: http://www.bbc.com/news/world-us-canada-39626959
Insert successful: http://www.bbc.com/news/business-39627135
<class 'pymysql.err.IntegrityError'> Error: http://www.bbc.com/news/technology-39625679
Insert successful: http://www.bbc.com/news/world-middle-east-39628860
Insert successful: http://www.bbc.com/news/world-africa-39619979
Insert successful: http://www.bbc.com/news/world-australia-39626389
Insert successful: http://www.bbc.com/news/world-africa-39629421
Insert successful: http://www.bbc.com/news/world-australia-39550820
Insert successful: http://www.bbc.com/news/world-us-canada-39624632
Insert successful: http://www.bbc.com/news/health-39627170
Insert successful: http://www.bbc.com/news/world-asia-39624508
I